<a href="https://colab.research.google.com/github/alxmzr/Colab/blob/main/Bollinger_Trading_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# A very Bollinger Trading Strategy (Python Tutorial)
# https://medium.com/@chris_42047/a-very-bollinger-trading-strategy-python-tutorial-2c6cc6a643ca

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/Colab Notebooks/BB-Trading-Strategy

In [ ]:
%%writefile requirements.txt
numpy
pandas
pandas_ta
ta
yfinance
backtesting

In [ ]:
!pip install -r requirements.txt

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
US_BUSINESS_DAY = CustomBusinessDay(calendar=USFederalHolidayCalendar())
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
US_BUSINESS_DAY = CustomBusinessDay(calendar=USFederalHolidayCalendar())
import pandas as pd
from backtesting import Strategy
from backtesting import Backtest
import yfinance as yf
import numpy as np
import os
from ta.volatility import BollingerBands
import pandas_ta as ta

In [ ]:
def load_data(symbol, interval):
    #  Download data
    # intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
    #  periods:  1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
    data = yf.download(tickers=symbol, period="1mo", interval=interval)
    df = pd.DataFrame(data)
    df['Date'] = pd.to_datetime(df.index)
    df.dropna(inplace=True)
    return df

def EMA(df_in, window):
    df = df_in.copy()
    ema_series = ta.ema(df['Close'], length=window)
    return ema_series
def RSI(df, rsi_period, _, __, ___):
    rsi_series = ta.rsi(df['Close'], window=rsi_period)
    return rsi_series
def BOLLINGER_LBAND(df, _, column_name, bollinger_window, bollinger_std):
    indicator_bb = BollingerBands(close=df[column_name], window=bollinger_window, window_dev=bollinger_std)
    return indicator_bb.bollinger_lband()
def BOLLINGER_HBAND(df, _, column_name, bollinger_window, bollinger_std):
    indicator_bb = BollingerBands(close=df[column_name], window=bollinger_window, window_dev=bollinger_std)
    return indicator_bb.bollinger_hband()

def calculate_trend(values):
    if len(values) == 0:
        return 0, 0
    x = np.arange(1, len(values) + 1, 1)
    y = np.array(values)
    #  Handle nan values
    x_new = x[~np.isnan(y)]
    y_new = y[~np.isnan(y)]
    m, c = np.polyfit(x_new, y_new, 1)
    return m, c

class VeryBollingerStrategy(Strategy):
    ema1_period = 1000
    rsi_period = 14
    rsi_lookback = 5
    bollinger_window = 20
    bollinger_std1 = 2.6
    bollinger_std2 = 1.6
    bollinger_std3 = 2.0
    last_trade_index = 0
    last_long_buy_price = 0
    long_hold = 0
    take_profit = 1 # percent
    stop_loss = 10 # percent
    i = 0
    def init(self):
        super().init()
        #  Calculate indicators
        self.ema1 = self.I(EMA, self.data.df, self.ema1_period)
        self.bollinger_close_lband = self.I(BOLLINGER_LBAND, self.data.df, None, 'Close', self.bollinger_window, self.bollinger_std1)
        self.bollinger_close_hband = self.I(BOLLINGER_HBAND, self.data.df, None, 'Close', self.bollinger_window, self.bollinger_std2)
        #  Calculate TIs
        self.data.df['RSI'] = RSI(df, self.rsi_period, None, None, None)
        self.rsi_bollinger_bands = self.I(lambda *args: (RSI(*args), BOLLINGER_LBAND(*args), BOLLINGER_HBAND(*args)),
                           self.data.df, self.rsi_period, "RSI", self.bollinger_window, self.bollinger_std3)
    def next(self):
        super().init()
        self.i += 1
        if self.i < self.ema1_period:
            return
        long_entry_signals = 0
        long_exit_signals = 0
        #  Long-term trend filter
        ema1_lb = self.ema1[-20:]
        ema1_m, _ = calculate_trend(ema1_lb)
        if ema1_m >= 0.01:
            long_entry_signals += 1
        #  Long entry
        if self.data.Close[-1] >= self.bollinger_close_lband[-1] and self.data.Close[-2] < self.bollinger_close_lband[-2]:
            long_entry_signals += 1
        rsi = self.rsi_bollinger_bands[0]
        rsi_lb = rsi[-self.rsi_lookback:]
        bollinger_rsi_lband = self.rsi_bollinger_bands[1]
        bollinger_rsi_lband_lb = bollinger_rsi_lband[-self.rsi_lookback:]
        for i in range(1, len(rsi_lb)):
            if rsi_lb[i] >= bollinger_rsi_lband_lb[i] and rsi_lb[i-1] < bollinger_rsi_lband_lb[i]:
                long_entry_signals += 1
                break
        #  Long exit
        if self.data.Close[-1] >= self.bollinger_close_hband[-1] and \
            self.data.Close[-1] >= self.last_long_buy_price * (1 + self.take_profit / 100):
            long_exit_signals += 1
        if self.data.Close[-1] <= self.last_long_buy_price * (1 - self.stop_loss/100):
            long_exit_signals += 1
        #  Go long
        if self.long_hold == 0 and long_entry_signals >= 3:
            self.buy()
            self.long_hold = 1
            self.last_long_buy_price = self.data.Close[-1]
        elif self.long_hold == 1 and long_exit_signals >= 1:
            # Close any existing long trades, and sell the asset
            self.position.close()
            self.long_hold = 0
            self.last_long_buy_price = 0

def run_backtest(df):
    # If exclusive orders (each new order auto-closes previous orders/position),
    # cancel all non-contingent orders and close all open trades beforehand
    bt = Backtest(df, VeryBollingerStrategy, cash=10000, commission=.00075, trade_on_close=True, exclusive_orders=False, hedging=False)
    stats = bt.run()
    print(stats)
    bt.plot()

if __name__ == '__main__':
    symbol = "ETH-USD"
    # intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
    interval = "15m"
    #  Load data
    df = load_data(symbol, interval)
    #  Run backtest
    run_backtest(df)

[*********************100%***********************]  1 of 1 completed
Start                     2022-08-16 14:45...
End                       2022-09-16 14:48...
Duration                     31 days 00:03:00
Exposure Time [%]                         0.0
Equity Final [$]                      10000.0
Equity Peak [$]                       10000.0
Return [%]                                0.0
Buy & Hold Return [%]               -7.480111
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade